In [1]:
import requests
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_input(day, year, save_dir="inputs"):
    """
    Fetches input for a specific Advent of Code day and saves it locally.

    :param day: The day of the Advent of Code (1-25).
    :param year: The Advent of Code year.
    :param save_dir: Directory to save the input file.
    :return: Path to the input file.
    """
    session_token = os.getenv("SESSION_TOKEN")
    if not session_token:
        raise ValueError("Session token not found in environment variables.")

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    headers = {
        "Cookie": f"session={session_token}",
        "User-Agent": "https://github.com/sai-pendyala/Advent-of-Code-2024",
    }

    # Create the directory if it doesn't exist
    Path(save_dir).mkdir(exist_ok=True)
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    # Fetch input
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(input_file, "w") as f:
            f.write(response.text.strip())
        print(f"Input for Day {day} saved to {input_file}")
        return input_file
    elif response.status_code == 404:
        print("Puzzle input is not available yet. Check after the problem is released.")
    else:
        print(f"Failed to fetch input: {response.status_code}")
    return None

In [2]:
def read_input(day, save_dir="inputs"):
    """
    Reads the content of the input file for a specific day of Advent of Code.

    :param day: The day of the Advent of Code (1-25).
    :param save_dir: Directory where the input file is saved (default is 'inputs').
    :return: Content of the input file as a string.
    """
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    if input_file.exists():
        with open(input_file, "r") as file:
            content = file.read().strip()  # Read and strip any surrounding whitespace
        print(f"Content of Day {day} input:\n{content}")
        return content
    else:
        print(f"Input file for Day {day} does not exist. Please fetch the input first.")
        return None

In [3]:
# Daily Workflow
from pathlib import Path
from datetime import datetime

# Dynamic Day and Year
now = datetime.utcnow()
DAY = now.day
YEAR = now.year

# Fetch input
fetch_input(DAY, YEAR)

# Read input
input_data = read_input(DAY)

/tmp/ipykernel_2047/4008538322.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Input for Day 5 saved to inputs/day05_input.txt
Content of Day 5 input:
86|99
69|32
69|42
77|99
77|72
77|38
21|57
21|89
21|65
21|47
47|53
47|75
47|62
47|69
47|38
95|72
95|16
95|94
95|15
95|89
95|53
51|94
51|95
51|53
51|77
51|15
51|16
51|57
49|27
49|17
49|97
49|29
49|87
49|38
49|11
49|55
31|86
31|77
31|34
31|94
31|26
31|85
31|79
31|36
31|57
94|77
94|91
94|99
94|62
94|53
94|61
94|56
94|69
94|26
94|83
35|94
35|99
35|91
35|29
35|97
35|89
35|49
35|56
35|47
35|86
35|27
53|62
53|42
53|69
53|76
53|59
53|27
53|49
53|72
53|11
53|89
53|26
53|91
24|94
24|95
24|15
24|47
24|64
24|65
24|91
24|35
24|49
24|16
24|75
24|51
24|89
75|32
75|91
75|42
75|99
75|11
75|62
75|97
75|72
75|83
75|59
75|53
75|56
75|27
75|29
89|73
89|62
89|97
89|99
89|83
89|38
89|56
89|15
89|29
89|87
89|55
89|27
89|76
89|11
89|17
87|33
87|11
87|57
87|34
87|51
87|95
87|45
87|36
87|31
87|64
87|98
87|35
87|85
87|55
87|21
87|32
45|75
45|86
45|16
45|57
45|72
45|99
45|49
45|53
45|64
45|51
45|26
45|15
45|95
45|77
45|69
45|59
45|89
42|27
42|6

# Day 1

In [7]:
lefts = []
rights = []
for line in input_data.split("\n"):
  left, right = line.strip().split()
  lefts.append(int(left))
  rights.append(int(right))

In [8]:
def get_total_distance(lefts, rights):
  total_distance = 0
  for left, right in zip(lefts, rights):
    total_distance += abs(left - right)
  return total_distance

get_total_distance(sorted(lefts), sorted(rights))

1889772

In [9]:
def get_similarity_score(lefts, rights):
  from collections import Counter
  similarity = 0
  right_counts = Counter(rights)
  for left in lefts:
    if left in right_counts:
      similarity += left * right_counts[left]
  return similarity

get_similarity_score(lefts, rights)

23228917

# Day 2

In [17]:
def is_safe(report):
    sign = 1
    if report[1] < report[0]:
        sign = -1

    for i in range(1, len(report)):
        diff = report[i] - report[i - 1]
        if sign * diff < 0 or not 1 <= abs(diff) <= 3:
            return False  
    return True

In [19]:
total_safe = 0
for line in input_data.split("\n"):
    check = is_safe(list(map(int, line.split())))
    total_safe += check

total_safe

549

In [24]:
total_safe2 = 0
for line in input_data.split("\n"):
    report = list(map(int, line.split()))
    for i in range(len(report)):
        first_half = report[:i] if i > 0 else []
        second_half = report[i + 1:] if i < len(report) - 1 else []
        check = is_safe(first_half + second_half)
        if check:
            total_safe2 += 1
            break

total_safe2

589

# Day 3

In [4]:
import re
pattern = r"mul\(\d+,\d+\)"
matches = re.findall(pattern, input_data)

total = 0
for match in matches:
    num1, num2 = map(int, match.strip("mul()").split(","))
    total += num1 * num2
total

155955228

In [5]:
import re
pattern = r"mul\(\d+,\d+\)|do\(\)|don't\(\)"
matches = re.findall(pattern, input_data)

total = 0
do = 1
for match in matches:
    if match == "do()":
        do = 1
    elif match == "don't()":
        do = 0
    else:
        num1, num2 = map(int, match.strip("mul()").split(","))
        total += num1 * num2 * do
total

100189366

# Day 4

In [6]:
grid = []
for line in input_data.split("\n"):
    grid.append(list(line))
m, n = len(grid), len(grid[0])

In [15]:
def dfs(curr, x, y, dx, dy):
    if curr == "XMAS":
        return 1
    if not (0 <= x < m and 0 <= y < n):
        return 0
    if len(curr) > 4:
        return 0
    curr += grid[x][y]
    return dfs(curr, x + dx, y + dy, dx, dy)

In [16]:
result = 0
for i in range(m):
    for j in range(n):
        if grid[i][j] == "X":
            for di, dj in [(-1, 0), (0, 1), (1, 0), (0, -1), (-1, 1), (1, 1), (1, -1), (-1, -1)]:
                result += dfs("", i, j, di, dj)

result

2462

In [24]:
result = 0
patterns = ["MAS", "SAM"]
for i in range(1, m - 1):
    for j in range(1, n - 1):
        if grid[i][j] == "A":
            diagonal1 = grid[i - 1][j - 1] + grid[i][j] + grid[i + 1][j + 1]
            diagonal2 = grid[i + 1][j - 1] + grid[i][j] + grid[i - 1][j + 1]
            # print(i, j, diagonal1, diagonal2)
            if diagonal1 in patterns and diagonal2 in patterns:
                result += 1
result

1877

# Day 5

In [13]:
def get_right_order(graph, degree):
    from collections import deque
    q = deque()
    for u, d in degree.items():
        if d == 0:
            q.append(u)
    right_order = []
    while q:
        node = q.popleft()
        right_order.append(node)
        for v in graph[node]:
            degree[v] -= 1
            if degree[v] == 0:
                q.append(v)
                
    return right_order

In [14]:
from collections import defaultdict

rules = defaultdict(set)
updates = []

for line in input_data.split("\n"):
    if line == '':
        continue 
    if '|' in line:
        u, v = line.split('|')
        rules[u].add(v)
    else:
        updates.append(line.split(','))

print(rules, updates)

defaultdict(<class 'set'>, {'86': {'87', '99', '76', '15', '97', '69', '49', '75', '61', '29', '91', '72', '89', '83', '26', '62', '56', '27', '53', '42', '38', '59', '32', '65'}, '69': {'87', '98', '76', '99', '33', '15', '97', '11', '17', '61', '29', '89', '83', '31', '62', '56', '27', '42', '38', '59', '55', '73', '32', '25'}, '77': {'87', '99', '76', '15', '97', '69', '49', '75', '61', '91', '29', '72', '89', '83', '26', '62', '56', '27', '53', '42', '38', '59', '86', '65'}, '21': {'77', '99', '15', '69', '49', '94', '75', '29', '91', '95', '72', '89', '26', '57', '53', '42', '64', '47', '59', '35', '51', '16', '86', '65'}, '47': {'77', '99', '15', '97', '69', '49', '94', '75', '29', '91', '72', '89', '83', '26', '62', '56', '53', '27', '42', '38', '59', '16', '86', '65'}, '95': {'77', '99', '15', '97', '69', '49', '94', '75', '29', '91', '72', '89', '26', '57', '56', '53', '42', '64', '47', '59', '35', '16', '86', '65'}, '51': {'77', '99', '15', '69', '49', '94', '75', '91', '29',

In [15]:
result1 = 0
result2 = 0
for update in updates:
    degree = {u: 0 for u in update} 
    graph = defaultdict(set)

    for u in update:
        graph[u] = rules[u] & set(update)
        for v in graph[u]:
            degree[v] += 1
    right_order = get_right_order(graph, degree)
    if right_order == update:
        result1 += int(right_order[(len(update) - 1) // 2])
    else:
        result2 += int(right_order[(len(update) - 1) // 2])
    
result1, result2

(4872, 5564)